In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

df=pd.read_csv('/content/drive/MyDrive/Dscover/메인프젝/dataset2.csv')

In [ ]:
#이직준비여부 컬럼의 값이 1 혹은 2가 아닌 행 제거
df = df[df['이직준비여부'].isin([1, 2])]

In [ ]:
#첫직장여부 == 1인 행들은 현직장과 첫직장 답변 일치하게 만듦
df1 = df[df['A76.현직장_첫직장여부'] == 1].copy()

#iloc을 사용하여 값 대체
df1.iloc[:, 121:153] = df1.iloc[:, 36:68].values

# 원래 데이터프레임의 해당 행들을 삭제하고, df1을 다시 합침
df = df[df['A76.현직장_첫직장여부'] != 1]
df = pd.concat([df, df1], ignore_index=True)

In [ ]:
#A13의 결측치는 비임금 근로자임. 따라서 410개 행 제거

# 각 컬럼별 결측값 수 확인
missing_values = df.isnull().sum()

# 결측값 수를 데이터프레임으로 변환하여 출력 (컬럼명을 열에, 결측값을 행에)
missing_values_df = pd.DataFrame(missing_values).transpose()
missing_values_df.columns = missing_values.index
missing_values_df.index = ['MissingValues']

missing_values_df

In [ ]:
#부양자녀 유무의 -1을 모두 2로 바꾼 후, 모든 부양자녀유무 2를 부양자녀수 0으로 만듦

# 각 컬럼별 결측값 수 확인
missing_values = df.isnull().sum()

# 결측값 수를 데이터프레임으로 변환하여 출력 (컬럼명을 열에, 결측값을 행에)
missing_values_df = pd.DataFrame(missing_values).transpose()
missing_values_df.columns = missing_values.index
missing_values_df.index = ['MissingValues']

# P3.부양자녀유무 열의 값이 -1인 행들을 2로 변경
df.loc[df['P3.부양자녀유무'] == -1, 'P3.부양자녀유무'] = 2

# P3.부양자녀유무 열의 값이 2인 경우 P3.부양자녀수 열의 값을 0으로 변경
df.loc[df['P3.부양자녀유무'] == 2, 'P3.부양자녀수'] = 0

In [ ]:
##경제적 지원을 하고 있는지 여부
#안함: 8974
#모름/무응답: 186
#합계: 9160 = 지원금액열의 결측치 개수
#따라서, 모름/무응답을 2로 통합 후 액수 결측치에 0 기입

# 각 컬럼별 결측값 수 확인
missing_values = df.isnull().sum()

# 결측값 수를 데이터프레임으로 변환하여 출력 (컬럼명을 열에, 결측값을 행에)
missing_values_df = pd.DataFrame(missing_values).transpose()
missing_values_df.columns = missing_values.index
missing_values_df.index = ['MissingValues']

# 'R3.가족경제적지원하는지여부' 값 중 -1을 2로 변경
df.loc[df['R3.가족경제적지원하는지여부'] == -1, 'R3.가족경제적지원하는지여부'] = 2

df.loc[df['R3.가족경제적지원하는지여부'] == 2, 'R3.월평균지원금액'] = 0

In [ ]:
#사업체 시, 도 결측치 29개 제거

# 각 컬럼별 결측값 수 확인
missing_values = df.isnull().sum()

# 결측값 수를 데이터프레임으로 변환하여 출력 (컬럼명을 열에, 결측값을 행에)
missing_values_df = pd.DataFrame(missing_values).transpose()
missing_values_df.columns = missing_values.index
missing_values_df.index = ['MissingValues']

# A6.현사업체_시도 열의 결측치가 있는 행 제거
df = df.dropna(subset=['A6.현사업체_시도'])

In [ ]:
#전공 소분류, 세분류 코드 column 제외
df=df.drop(columns=['전공_소분류 CODE', '전공 세분류 CODE 혹은 전공명'])

#졸업 년월 하나로 합치기
import datetime

for i in range(len(df)) :
  df['졸업년'][i]=datetime.date(year=df['졸업년'][i],month=df['졸업월'][i], day=1)

#현일자리 시작년월 하나로 합치기
for i in range(len(df)) :
  if df['A1.현일자리_시작월'][i]!=-1:
    df['A1.현일자리_시작년'][i]=datetime.date(year=int(df['A1.현일자리_시작년'][i]),month=int(df['A1.현일자리_시작월'][i]), day=1)
  else:
    df['A1.현일자리_시작년'][i]=datetime.date(year=int(df['A1.현일자리_시작년'][i]),month=1, day=1)

df=df.drop(columns=['졸업월','A1.현일자리_시작월'])

<ipython-input-3-f651468167eb>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['졸업년'][i]=datetime.date(year=df['졸업년'][i],month=df['졸업월'][i], day=1)
<ipython-input-3-f651468167eb>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['A1.현일자리_시작년'][i]=datetime.date(year=int(df['A1.현일자리_시작년'][i]),month=int(df['A1.현일자리_시작월'][i]), day=1)


In [ ]:
#출근시간 하나로 합치기 (분단위)
for i in range(len(df)) :
  df['A7-1.현직장_출근hour'][i]=df['A7-1.현직장_출근hour'][i]*60+df['A7-1.현직장_출근min'][i]
df=df.drop(columns=['A7-1.현직장_출근min'])

#출근시간 모르겠음(-1) -> 음수가 아닌 값들의 평균으로 처리
non_negative_mean = df[df['A7-1.현직장_출근hour'] >= 0]['A7-1.현직장_출근hour'].mean()
df.loc[df['A7-1.현직장_출근hour'] < 0, 'A7-1.현직장_출근hour'] = non_negative_mean

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
<ipython-input-4-74b631703e03>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['A7-1.현직장_출근hour'][i]=df['A7-1.현직장_출근hour'][i]*60+df['A7-1.현직장_출근min'][i]
<ipython-input-4-74b631703e03>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['A7-1.현직장_출근hour'][i]=df['A7-1.현직장_출근hour'][i]*60+df['A7-1.현직장_출근min'][i]
<ipython-input-4-74b631703e03>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['A7-1.

In [ ]:
#한달 감정 긍정/부정 두 컬럼으로 종합
for i in range(len(df)) :
  df['Q9.한달간감정_즐거운'][i]=df['Q9.한달간감정_즐거운'][i]+df['Q9.한달간감정_행복한'][i]+df['Q9.한달간감정_편안한'][i]
for i in range(len(df)) :
  df['Q9.한달간감정_짜증나는'][i]=(-df['Q9.한달간감정_짜증나는'][i])+(-df['Q9.한달간감정_부정적인'][i])+(-df['Q9.한달간감정_무기력한'][i])
df[df.columns[72:80]]

df=df.drop(columns=["Q9.한달간감정_행복한",	"Q9.한달간감정_편안한"])
df=df.drop(columns=['Q9.한달간감정_부정적인','Q9.한달간감정_무기력한'])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
<ipython-input-5-256e1e310707>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Q9.한달간감정_짜증나는'][i]=(-df['Q9.한달간감정_짜증나는'][i])+(-df['Q9.한달간감정_부정적인'][i])+(-df['Q9.한달간감정_무기력한'][i])
<ipython-input-5-256e1e310707>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Q9.한달간감정_짜증나는'][i]=(-df['Q9.한달간감정_짜증나는'][i])+(-df['Q9.한달간감정_부정적인'][i])+(-df['Q9.한달간감정_무기력한'][i])
<ipython-input-5-256e1e310707>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

In [ ]:
#소득액 결측치 평균으로 처리
df['R1.전년도소득액_근로·사업소득액']=df['R1.전년도소득액_근로·사업소득액'].fillna(df['R1.전년도소득액_근로·사업소득액'].mean())
#기타응답 결측치 병합
df.loc[df['R1.전년도소득액_그외사회보험(기타응답)'] == 'No', 'R1.전년도소득액_그외사회보험(기타응답)'] = '모름/무응답'

In [ ]:
#급여형태에 다른 근로소득 -> 월평균 소득으로 대체
df=df.drop(columns=['A32.현직장_급여형태',	'A32.현직장_급여형태별근로소득','A32-1.현직장_초임급여형태', 'A32-1.현직장_초임급여형태별소득'])
df=df.drop(columns=['D30.첫직장_급여형태','D30.첫직장_급여형태별근로소득','D30-1.첫직장_초임급여형태','D30-1.첫직장_초임급여형태별소득'])

In [ ]:
#주평균 운동시간 이상치(주 100시간 운동함->이상치로 판단) 평균값으로 대체
replace_mean = df[df['Q2.주평균운동시간']<=20]['Q2.주평균운동시간'].mean()
df.loc[df['Q2.주평균운동시간'] > 20, 'Q2.주평균운동시간'] = replace_mean

In [ ]:
#수치형 변수 정규화

col_names=['A7-1.현직장_출근hour','A32.현직장_월평균근로소득','A32-1.현직장_초임월평균소득','Q2.주평균운동시간', 'Q3.일평균수면시간','Q9.한달간감정_즐거운', 'Q9.한달간감정_짜증나는','P3.부양자녀수','P6.거주가구원수','R1.전년도소득액_육아휴직급여액','R1.전년도소득액_ 산전후휴가급여액','R1.전년도소득액_실업급여액','R1.전년도소득액_그외사회보험액','R1.전년도소득액_공적이전소득액','R1.전년도소득액_경제적지원액',
           'R1.전년도소득액_금융,부동산소득액','R1.전년도소득액_금융,부동산소득대출액','R1.전년도소득액_기타소득액','R2.전년도월평균생활비지출액','R2-1.전년도월평균주거비지출액','R2-2.전년도월평균공교육비지출액','R2-2.전년도월평균사교육비지출액','R3.월평균지원금액'
           ,'D30.첫직장_월평균근로소득','D30-1.첫직장_초임월평균소득']

from sklearn.preprocessing import MinMaxScaler

# Min-Max 스케일러 인스턴스 생성
scaler = MinMaxScaler()

# col_names 리스트에 있는 각 컬럼에 대해 정규화 수행
for col in col_names:
    # 컬럼을 2차원 배열로 변환 (scaler가 2D 데이터를 요구하기 때문)
    df[col] = scaler.fit_transform(df[[col]])

In [ ]:
#근무기간(설문조사 날짜 - 현일자리 시작날짜) 컬럼 생성
from datetime import datetime
# 기준 날짜 설정
reference_date = datetime(2020, 9, 1)

df['A1.현일자리_시작년'] = pd.to_datetime(df['A1.현일자리_시작년'])

# 근무 기간 계산
df['근무기간'] = (reference_date - df['A1.현일자리_시작년']).dt.days
df['근무기간'] = scaler.fit_transform(df[['근무기간']])

#정규화
df['근무기간'].describe()

#현일자리 시작년 컬럼 제거
df=df.drop(columns=['A1.현일자리_시작년'])

count    11419.000000
mean         0.035162
std          0.037790
min          0.000000
25%          0.014256
50%          0.033160
75%          0.044782
max          1.000000
Name: 근무기간, dtype: float64

In [ ]:
#현직장만족도-첫직장만족도 비교 후 첫직장만족도 컬럼 대체

#'만족도'가 들어가는 컬럼명 출력
satisfaction_columns = df.columns[df.columns.str.contains('만족도')]
filtered_df = df[satisfaction_columns]

print(filtered_df.columns)

#만족도 비교한 값으로 대체
for i in range(14):
  df[filtered_df.columns[i+19]] = df[filtered_df.columns[i]] - df[filtered_df.columns[i+19]]

Index(['A33.현직장만족도_임금,소득', 'A33.현직장만족도_고용안정성', 'A33.현직장만족도_직무내용',
       'A33.현직장만족도_근무환경', 'A33.현직장만족도_근로시간,일수', 'A33.현직장만족도_개인발전가능성',
       'A33.현직장만족도_인간관계', 'A33.현직장만족도_복리후생제도', 'A33.현직장만족도_인사체계,승진제도',
       'A33.현직장만족도_일의사회적평판', 'A33.현직장만족도_일의자율성,권한', 'A33.현직장만족도_일자리사회적평판',
       'A33.현직장만족도_적성,흥미일치', 'A33.현직장만족도_직무관련교육,훈련', 'A33-1.현직장전반만족도',
       'A33-2.업무전반만족도', 'Q8.삶의만족도_개인적측면', 'Q8.삶의만족도_관계적측면', 'Q8.삶의만족도_소속집단만족도',
       'D31. 첫직장만족도_임금,소득', 'D31.첫직장만족도_고용안정성', 'D31.첫직장만족도_직무내용',
       'D31.첫직장만족도_근무환경', 'D31. 첫직장만족도_근로시간,일수', 'D31.첫직장만족도_개인발전가능성',
       'D31.첫직장만족도_인관관계', 'D31.첫직장만족도_복리후생제도', 'D31.첫직장만족도_인사체계,승진제도',
       'D31.첫직장만족도_일의사회적평판', 'D31.첫직장만족도_일의자율성,권한', 'D31.첫직장만족도_일자리사회적평판',
       'D31.첫직장만족도_적성, 흥미일치', 'D31.첫직장만족도_직무관련교육,훈련', 'D31-1.첫직장전반만족도',
       'D31-2.첫직장업무전반만족도'],
      dtype='object')


In [ ]:
#현/전직장 비교한 컬럼 이름 변경
df.rename(columns={'D31. 첫직장만족도_임금,소득': '현/전_임금,소득_만족도비교'}, inplace=True)
df.rename(columns={'D31.첫직장만족도_고용안정성': '현/전_고용안정성_만족도비교'}, inplace=True)
df.rename(columns={'D31.첫직장만족도_직무내용': '현/전_직무내용_만족도비교'}, inplace=True)
df.rename(columns={'D31.첫직장만족도_근무환경': '현/전_근무환경_만족도비교'}, inplace=True)
df.rename(columns={'D31.첫직장만족도_근로시간,일수': '현/전_근로시간,일수_만족도비교'}, inplace=True)
df.rename(columns={'D31.첫직장만족도_개인발전가능성': '현/전_개인발전가능성_만족도비교'}, inplace=True)
df.rename(columns={'D31.첫직장만족도_인관관계': '현/전_인간관계_만족도비교'}, inplace=True)
df.rename(columns={'D31.첫직장만족도_복리후생제도': '현/전_복리후생제도_만족도비교'}, inplace=True)
df.rename(columns={'D31.첫직장만족도_인사체계,승진제도': '현/전_인사체계,승진제도_만족도비교'}, inplace=True)
df.rename(columns={'D31.첫직장만족도_일의사회적평판': '현/전_일의사회적평판_만족도비교'}, inplace=True)
df.rename(columns={'D31.첫직장만족도_일의자율성,권한': '현/전_일의자율성,권한_만족도비교'}, inplace=True)
df.rename(columns={'D31.첫직장만족도_일자리사회적평판': '현/전_일자리사회적평판_만족도비교'}, inplace=True)
df.rename(columns={'D31.첫직장만족도_적성, 흥미일치': '현/전_적성,흥미일치_만족도비교'}, inplace=True)
df.rename(columns={'D31.첫직장만족도_직무관련교육,훈련': '현/전_직무관련교육,훈련'}, inplace=True)
df.rename(columns={'D31.첫직장만족도_직무관련교육,훈련': '현/전_직무관련교육,훈련'}, inplace=True)

In [ ]:
#현/전 만족도 비교 추가
df['D31-1.첫직장전반만족도'] = df['A33-1.현직장전반만족도'] - df['D31-1.첫직장전반만족도']
df['D31-2.첫직장업무전반만족도'] = df['A33-2.업무전반만족도'] - df['D31-2.첫직장업무전반만족도']
df.rename(columns={'D31-1.첫직장전반만족도': '현/전_전반만족도비교'}, inplace=True)
df.rename(columns={'D31-2.첫직장업무전반만족도': '현/전_업무전반만족도비교'}, inplace=True)

In [ ]:
#필요없는 컬럼 drop
df=df.drop(columns=['A4.현직장_대분류(2018)','A4.현직장_중분류(2018)',	'A4.현직장_소분류(2018)',	'A3.현직장_세분류(2018)'])
df=df.drop(columns=['A3.현직장_소분류'])
df=df.drop(columns=['전공 세분류명 (2017g~ 세분류명 별도변수로 분리)','A3.현직장_중분류','R1.전년도소득액_그외사회보험(기타응답)'
,'D32.첫직장_교육,일수준일치정도','D33.첫직장_기술수준일치정도','D34.첫직장_전공일치정도','D37.첫직장_국민연금가입여부','D37.첫직장_특수직역연금가입여부','D37.첫직장_건강보험가입여부',
'D37.첫직장_고용보험가입여부','D37.첫직장_산재보험가입여부','D38.첫직장_노동조합존재여부','D43.첫직장_일자리제의경험'])
df=df.drop(columns=['전공_소분류명 (2017g~ 소분류명 별도변수로 분리)'])

#현직장이 첫직장인 경우 결측치 = 1 로 바꾸기
df.loc[df['A76-1.현직장_n번째'] ==0.0, 'A76-1.현직장_n번째'] = 1.0

In [ ]:
#삶의 만족도 한 컬럼으로 병합
for i in range(len(df)) :
  df['Q8.삶의만족도_개인적측면'][i]=df['Q8.삶의만족도_개인적측면'][i]+df['Q8.삶의만족도_관계적측면'][i]+df['Q8.삶의만족도_소속집단만족도'][i]
df.rename(columns={'Q8.삶의만족도_개인적측면': 'Q8.삶의만족도'}, inplace=True)
df=df.drop(columns=['Q8.삶의만족도_관계적측면','Q8.삶의만족도_소속집단만족도'])

df.rename(columns={'Q9.한달간감정_즐거운': 'Q9.한달간감정_긍정'}, inplace=True)
df.rename(columns={'Q9.한달간감정_짜증나는': 'Q9.한달간감정_부정'}, inplace=True)

#이혼, 사별의 경우 미혼 case에 포함
df.loc[df['P1.혼인여부'].isin([3, 4]), 'P1.혼인여부'] = 1

#부모님 자산규모 없을 경우에 -1로 포함
df.loc[df['P12.부모님자산규모'] == 7, 'P12.부모님자산규모'] = -1
df.loc[df['P12.부모님자산규모'] == 8, 'P12.부모님자산규모'] = -1

#생활비충당방식 이상치 처리
df.loc[df['R4.생활비충당방식'] == 7, 'R4.생활비충당방식'] = -1
df.loc[df['R4.생활비충당방식'] == -3, 'R4.생활비충당방식'] = -1

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
<ipython-input-173-22a55937523d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Q8.삶의만족도_개인적측면'][i]=df['Q8.삶의만족도_개인적측면'][i]+df['Q8.삶의만족도_관계적측면'][i]+df['Q8.삶의만족도_소속집단만족도'][i]
<ipython-input-173-22a55937523d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Q8.삶의만족도_개인적측면'][i]=df['Q8.삶의만족도_개인적측면'][i]+df['Q8.삶의만족도_관계적측면'][i]+df['Q8.삶의만족도_소속집단만족도'][i]
<ipython-input-173-22a55937523d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Min-Max 스케일러 인스턴스 생성
scaler = MinMaxScaler()

#정규화해야할 컬럼
scale_cols=['A76-1.현직장_n번째','A33.현직장만족도_임금,소득','A33.현직장만족도_고용안정성','A33.현직장만족도_직무내용','A33.현직장만족도_근무환경','A33.현직장만족도_근로시간,일수',
'A33.현직장만족도_개인발전가능성','A33.현직장만족도_인간관계','A33.현직장만족도_복리후생제도','A33.현직장만족도_인사체계,승진제도','A33.현직장만족도_일의사회적평판','A33.현직장만족도_일의자율성,권한',
'A33.현직장만족도_일자리사회적평판','A33.현직장만족도_적성,흥미일치','A33.현직장만족도_직무관련교육,훈련','A33-1.현직장전반만족도','A33-2.업무전반만족도','A34.현직장_교육,일수준일치정도',
'A35.현직장_기술수준일치정도','A36.현직장_전공일치정도','Q8.삶의만족도','Q9.한달간감정_긍정','Q9.한달간감정_부정','R1.전년도소득액_근로·사업소득액','현/전_임금,소득_만족도비교',
'현/전_고용안정성_만족도비교','현/전_직무내용_만족도비교','현/전_근무환경_만족도비교','D31. 첫직장만족도_근로시간,일수','현/전_개인발전가능성_만족도비교','현/전_인간관계_만족도비교',
'현/전_복리후생제도_만족도비교','현/전_인사체계,승진제도_만족도비교','현/전_일의사회적평판_만족도비교','현/전_일의자율성,권한_만족도비교','현/전_일자리사회적평판_만족도비교',
'현/전_적성,흥미일치_만족도비교','현/전_직무관련교육,훈련','현/전_전반만족도비교','현/전_업무전반만족도비교']

# col_names 리스트에 있는 각 컬럼에 대해 정규화 수행
for col in scale_cols:
    # 컬럼을 2차원 배열로 변환 (scaler가 2D 데이터를 요구하기 때문)
    print(col)
    df[col] = scaler.fit_transform(df[[col]])

df.rename(columns={'D31. 첫직장만족도_근로시간,일수': '현/전_근로시간,일수_만족도비교'}, inplace=True)

df=df.drop(columns=['R1.전년도소득액_기타소득'])

In [ ]:
#거주지와 사업체 코드가 일치하면 0, 안일치하면 1
df['거주지,사업체_시도일치'] = np.where(df['P8.거주지시도'] == df['A6.현사업체_시도'], 0, 1)
df=df.drop(columns=['P8.거주지시도','A6.현사업체_시도'])

In [ ]:
#전처리 완료한 df csv로 내보내기
df.to_csv('dataset7.csv')